In [10]:
# ============================================================
# 🧠 Toss Document Classification: v10_b4_vit_ensemble (Advanced)
# ============================================================

import os, random, copy, cv2, timm, torch
import pandas as pd, numpy as np
import torch.nn as nn
import torch.nn.functional as F
from albumentations.pytorch import ToTensorV2
import albumentations as A
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from timm.loss import LabelSmoothingCrossEntropy



In [11]:
# ============================================================
# 0️⃣ Global Config
# ============================================================
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BASE_DIR = "/data/ephemeral/home/data"
TRAIN_META = os.path.join(BASE_DIR, "meta_stage0_10_1_train_v10.csv")
TEST_DIR = os.path.join(BASE_DIR, "processed", "stage0_10_1_test_v10")
SAMPLE_SUB = os.path.join(BASE_DIR, "raw", "sample_submission.csv")
OUT_DIR = "./runs_v10_b4_vit_ensemble"
os.makedirs(OUT_DIR, exist_ok=True)

SPECIAL_CLASSES = {3, 7, 14}

# --- EfficientNet-B4 ---
CFG_B4 = dict(
    model_name="tf_efficientnet_b4_ns",
    num_classes=17,
    img_size=380,
    batch_size=32,
    lr=3e-4,
    weight_decay=1e-4,
    epochs=50,
    patience=10,
    valid_ratio=0.8,
    num_workers=4,
    tta_times=20,
)

# --- ViT (for ensemble) ---
CFG_VIT = dict(
    model_name="vit_base_patch16_384",
    num_classes=17,
    img_size=384,
    batch_size=16,
    lr=3e-4,
    weight_decay=1e-4,
    epochs=40,
    patience=8,
    valid_ratio=0.8,
    num_workers=4,
    tta_times=20,
)



In [12]:
# ============================================================
# 1️⃣ Dataset
# ============================================================
class V10ImageDataset(Dataset):
    def __init__(self, csv_path, transform=None, oversample=False):
        df = pd.read_csv(csv_path)
        if oversample:
            df = self._oversample(df)
        self.df = df
        self.transform = transform

    def _oversample(self, df):
        factors = {3: 2, 7: 2, 14: 3}
        parts = [df]
        for cls, f in factors.items():
            sub = df[df["target"] == cls]
            if len(sub) > 0 and f > 1:
                parts.append(sub.loc[sub.index.repeat(f - 1)])
        out = pd.concat(parts, axis=0).reset_index(drop=True)
        print("✅ Oversample 완료:", {k: (out['target'] == k).sum() for k in factors})
        return out

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = cv2.imread(row["filepath"])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = row["target"]
        if self.transform:
            img = self.transform(image=img)["image"]
        return img, int(label)


class TestImageDataset(Dataset):
    def __init__(self, sample_csv, test_dir, transform):
        if isinstance(sample_csv, str):
            self.df = pd.read_csv(sample_csv)
        elif isinstance(sample_csv, pd.DataFrame):
            self.df = sample_csv.copy()
        else:
            raise ValueError("❌ sample_csv must be path or DataFrame")

        self.test_dir = test_dir
        self.transform = transform
        self.df["ID"] = self.df["ID"].apply(lambda x: x if str(x).endswith(".jpg") else f"{x}.jpg")

    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx]["ID"]
        path = os.path.join(self.test_dir, img_id)
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(image=img)["image"]
        return img, img_id



In [13]:
# ============================================================
# 2️⃣ Transform (Noise 중심 TTA 포함)
# ============================================================
def get_transform(img_size, tta=False):
    base = [
        A.LongestMaxSize(max_size=img_size, always_apply=True),
        A.PadIfNeeded(min_height=img_size, min_width=img_size,
                      border_mode=cv2.BORDER_CONSTANT, value=(255, 255, 255)),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
    if not tta:
        return A.Compose([
            A.RandomRotate90(p=0.5),
            A.OneOf([A.HorizontalFlip(p=0.5),
                     A.VerticalFlip(p=0.5)], p=0.5),
            A.RandomBrightnessContrast(p=0.3),
            A.GaussNoise(var_limit=(10.0, 40.0), p=0.3),
            A.MotionBlur(p=0.2),
            A.ImageCompression(quality_lower=70, quality_upper=100, p=0.2),
            A.MedianBlur(blur_limit=3, p=0.2),
        ] + base)
    else:
        return A.Compose([
            A.RandomRotate90(p=1.0),
            A.OneOf([A.HorizontalFlip(p=0.5),
                     A.VerticalFlip(p=0.5)], p=0.5),
            A.GaussNoise(var_limit=(10.0, 40.0), p=0.3),
            A.MotionBlur(p=0.2),
        ] + base)

# ============================================================
# 3️⃣ Train / Eval / Infer
# ============================================================
def create_model(model_name, num_classes, drop=0.2):
    model = timm.create_model(model_name, pretrained=True,
                              num_classes=num_classes, drop_rate=drop)
    model.to(device)
    return model

def train_one_epoch(model, loader, criterion, optimizer, scaler):
    model.train(); total_loss=0; preds=[]; tgts=[]
    for imgs, lbls in tqdm(loader, desc="[Train]", leave=False):
        imgs, lbls = imgs.to(device), lbls.to(device)
        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast('cuda'):
            out = model(imgs); loss = criterion(out, lbls)
        scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
        total_loss += loss.item()
        preds += out.argmax(1).cpu().tolist(); tgts += lbls.cpu().tolist()
    return total_loss/len(loader), accuracy_score(tgts,preds), f1_score(tgts,preds,average='macro')

@torch.no_grad()
def eval_one_epoch(model, loader, criterion):
    model.eval(); total_loss=0; preds=[]; tgts=[]
    for imgs, lbls in tqdm(loader, desc="[Valid]", leave=False):
        imgs, lbls = imgs.to(device), lbls.to(device)
        with torch.amp.autocast('cuda'):
            out = model(imgs); loss = criterion(out, lbls)
        total_loss += loss.item()
        preds += out.argmax(1).cpu().tolist(); tgts += lbls.cpu().tolist()
    return total_loss/len(loader), accuracy_score(tgts,preds), f1_score(tgts,preds,average='macro')

@torch.no_grad()
def infer_probs(model, loader):
    model.eval(); all_probs=[]; all_ids=[]
    for imgs, ids in tqdm(loader, desc="[Infer]", leave=False):
        imgs = imgs.to(device)
        with torch.amp.autocast('cuda'):
            out = F.softmax(model(imgs), dim=1)
        all_probs.append(out.cpu().numpy()); all_ids.extend(ids)
    return np.concatenate(all_probs), all_ids



In [14]:
# ============================================================
# 4️⃣ Train Loop
# ============================================================
def train_loop(cfg, train_ds, valid_ds, model_name, model_tag):
    train_loader = DataLoader(train_ds, batch_size=cfg["batch_size"], shuffle=True,
                              num_workers=cfg["num_workers"], pin_memory=True)
    valid_loader = DataLoader(valid_ds, batch_size=cfg["batch_size"], shuffle=False,
                              num_workers=cfg["num_workers"], pin_memory=True)

    model = create_model(model_name, cfg["num_classes"])
    weights = torch.tensor([1.5 if i in [3,7,14] else 1.0 for i in range(cfg["num_classes"])],
                           dtype=torch.float32).to(device)
    criterion = nn.CrossEntropyLoss(weight=weights)
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg["lr"],
                                  weight_decay=cfg["weight_decay"])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cfg["epochs"], eta_min=1e-6)
    scaler = torch.amp.GradScaler('cuda')

    best_f1, counter = -1, 0
    best_path = os.path.join(OUT_DIR, f"model_{model_tag}_best.pt")

    for e in range(1, cfg["epochs"]+1):
        print(f"\n==== [{model_tag}] Epoch {e}/{cfg['epochs']} ====")
        tr_l, tr_a, tr_f = train_one_epoch(model, train_loader, criterion, optimizer, scaler)
        va_l, va_a, va_f = eval_one_epoch(model, valid_loader, criterion)
        scheduler.step()
        print(f"[Train] loss={tr_l:.4f} acc={tr_a:.4f} f1={tr_f:.4f}")
        print(f"[Valid] loss={va_l:.4f} acc={va_a:.4f} f1={va_f:.4f}")
        if va_f > best_f1:
            best_f1 = va_f; counter = 0
            torch.save(model.state_dict(), best_path)
            print(f"✅ Best updated: F1={best_f1:.5f}")
        else:
            counter += 1
            print(f"⏳ No improve ({counter}/{cfg['patience']})")
            if counter >= cfg["patience"]:
                print("🛑 Early stop"); break
    return best_path, best_f1


In [15]:

# ============================================================
# 5️⃣ Inference (b4 + vit soft voting)
# ============================================================
@torch.no_grad()
def ensemble_infer(b4_ckpt, vit_ckpt):
    print("\n🔄 Ensemble Inference 시작")
    models_cfg = [
        (CFG_B4, b4_ckpt),
        (CFG_VIT, vit_ckpt),
    ]
    model_probs = []

    for cfg, ckpt in models_cfg:
        model = create_model(cfg["model_name"], cfg["num_classes"])
        model.load_state_dict(torch.load(ckpt))
        model.eval()

        base_ds = TestImageDataset(SAMPLE_SUB, TEST_DIR, get_transform(cfg["img_size"]))
        base_loader = DataLoader(base_ds, batch_size=cfg["batch_size"], shuffle=False)
        probs, ids = infer_probs(model, base_loader)

        tta_sum = np.zeros_like(probs)
        for i in range(cfg["tta_times"]):
            print(f"[{cfg['model_name']}] TTA round {i+1}/{cfg['tta_times']}")
            tta_ds = TestImageDataset(SAMPLE_SUB, TEST_DIR, get_transform(cfg["img_size"], tta=True))
            tta_loader = DataLoader(tta_ds, batch_size=cfg["batch_size"], shuffle=False)
            tta_sum += infer_probs(model, tta_loader)[0]
        final_probs = (probs + tta_sum / cfg["tta_times"]) / 2
        model_probs.append(final_probs)

    # ✅ Soft voting (자동 가중치 계산)
    corr = np.corrcoef(model_probs[0].ravel(), model_probs[1].ravel())[0,1]
    w1, w2 = (1-corr), (1+corr)
    print(f"📊 Auto ensemble weights → b4:{w1:.3f}, vit:{w2:.3f}")

    final = (w1*model_probs[0] + w2*model_probs[1]) / (w1 + w2)
    preds = final.argmax(1)

    out = pd.DataFrame({"ID": ids, "target": preds})
    path = os.path.join(OUT_DIR, "submission_v13_b4_vit.csv")
    out.to_csv(path, index=False)
    print(f"✅ 결과 저장: {path}")
    return path



In [7]:
# ============================================================
# 6️⃣ Main Routine
# ============================================================
def main():
    full = V10ImageDataset(TRAIN_META, get_transform(CFG_B4["img_size"]), oversample=True)
    n = int(len(full) * (1 - CFG_B4["valid_ratio"]))
    tr, v = random_split(full, [len(full) - n, n], generator=torch.Generator().manual_seed(SEED))
    v = copy.deepcopy(v); v.dataset.transform = get_transform(CFG_B4["img_size"], tta=False)

    b4_ckpt, f1_b4 = train_loop(CFG_B4, tr, v, CFG_B4["model_name"], "b4")
    vit_ckpt, f1_vit = train_loop(CFG_VIT, tr, v, CFG_VIT["model_name"], "vit")

    print(f"\n[완료] B4_F1={f1_b4:.4f}, ViT_F1={f1_vit:.4f}")
    ensemble_infer(b4_ckpt, vit_ckpt)

if __name__ == "__main__":
    main()


/tmp/ipykernel_437905/1902108288.py:6: UserWarning: Argument(s) 'always_apply' are not valid for transform MaxSizeTransform
  A.LongestMaxSize(max_size=img_size, always_apply=True),
/tmp/ipykernel_437905/1902108288.py:7: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=img_size, min_width=img_size,
/tmp/ipykernel_437905/1902108288.py:19: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 40.0), p=0.3),
/tmp/ipykernel_437905/1902108288.py:21: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=70, quality_upper=100, p=0.2),


✅ Oversample 완료: {3: np.int64(12672), 7: np.int64(12928), 14: np.int64(9792)}


/tmp/ipykernel_437905/1902108288.py:6: UserWarning: Argument(s) 'always_apply' are not valid for transform MaxSizeTransform
  A.LongestMaxSize(max_size=img_size, always_apply=True),
/tmp/ipykernel_437905/1902108288.py:7: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=img_size, min_width=img_size,
/tmp/ipykernel_437905/1902108288.py:19: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 40.0), p=0.3),
/tmp/ipykernel_437905/1902108288.py:21: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=70, quality_upper=100, p=0.2),
/data/ephemeral/home/py310/lib/python3.10/site-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(



==== [b4] Epoch 1/50 ====


[Train] loss=0.2565 acc=0.9186 f1=0.9333
[Valid] loss=0.0358 acc=0.9886 f1=0.9908
✅ Best updated: F1=0.99077

==== [b4] Epoch 2/50 ====


...

[Train] loss=0.0002 acc=0.9999 f1=0.9999
[Valid] loss=0.0005 acc=0.9999 f1=0.9999
⏳ No improve (7/10)

==== [b4] Epoch 38/50 ====


[Train] loss=0.0001 acc=0.9999 f1=0.9999
[Valid] loss=0.0005 acc=0.9999 f1=0.9999
⏳ No improve (8/10)

==== [b4] Epoch 39/50 ====


[Train] loss=0.0001 acc=1.0000 f1=1.0000
[Valid] loss=0.0002 acc=0.9999 f1=0.9999
⏳ No improve (9/10)

==== [b4] Epoch 40/50 ====


[Train] loss=0.0001 acc=1.0000 f1=1.0000
[Valid] loss=0.0002 acc=0.9999 f1=0.9999
⏳ No improve (10/10)
🛑 Early stop

==== [vit] Epoch 1/40 ====


AssertionError: Input height (380) doesn't match model (384).

In [16]:
# ============================================================
# 🧩 Resume from B4 checkpoint & train ViT + B5 + ensemble
# ============================================================
def resume_main():
    print("🚀 Resume mode: b4 건너뛰고 vit + b5 + ensemble 실행")

    # 1️⃣ b4 체크포인트 불러오기
    b4_ckpt = os.path.join(OUT_DIR, "model_b4_best.pt")
    if not os.path.exists(b4_ckpt):
        raise FileNotFoundError("❌ model_b4_best.pt not found. b4 학습을 먼저 완료해야 합니다!")

    # 2️⃣ ViT 전용 데이터셋 (img_size=384)
    full_vit = V10ImageDataset(TRAIN_META, get_transform(CFG_VIT["img_size"]), oversample=True)
    n_vit = int(len(full_vit) * (1 - CFG_VIT["valid_ratio"]))
    tr_vit, v_vit = random_split(full_vit, [len(full_vit) - n_vit, n_vit],
                                 generator=torch.Generator().manual_seed(SEED))
    v_vit = copy.deepcopy(v_vit)
    v_vit.dataset.transform = get_transform(CFG_VIT["img_size"], tta=False)

    # 3️⃣ ViT 학습
    vit_ckpt, f1_vit = train_loop(CFG_VIT, tr_vit, v_vit, CFG_VIT["model_name"], "vit")

    # 4️⃣ (선택) b5 서브모델 추가
    CFG_B5 = dict(
        model_name="tf_efficientnet_b5_ns",
        num_classes=17,
        img_size=456,
        batch_size=16,
        lr=3e-4,
        weight_decay=1e-4,
        epochs=40,
        patience=8,
        valid_ratio=0.8,
        num_workers=4,
        tta_times=20,
    )

    full_b5 = V10ImageDataset(TRAIN_META, get_transform(CFG_B5["img_size"]), oversample=True)
    n_b5 = int(len(full_b5) * (1 - CFG_B5["valid_ratio"]))
    tr_b5, v_b5 = random_split(full_b5, [len(full_b5) - n_b5, n_b5],
                               generator=torch.Generator().manual_seed(SEED))
    v_b5 = copy.deepcopy(v_b5)
    v_b5.dataset.transform = get_transform(CFG_B5["img_size"], tta=False)

    b5_ckpt, f1_b5 = train_loop(CFG_B5, tr_b5, v_b5, CFG_B5["model_name"], "b5")

    # 5️⃣ 3모델 앙상블 (b4 + vit + b5)
    print(f"\n[완료] B4✓ | ViT_F1={f1_vit:.4f} | B5_F1={f1_b5:.4f}")
    ensemble_3model_infer(b4_ckpt, vit_ckpt, b5_ckpt)



In [17]:

# ============================================================
# 3-model Ensemble with TTA
# ============================================================
@torch.no_grad()
def ensemble_3model_infer(b4_ckpt, vit_ckpt, b5_ckpt):
    print("\n🔄 3-model Ensemble Inference 시작")

    models_cfg = [
        (CFG_B4, b4_ckpt),
        (CFG_VIT, vit_ckpt),
        (CFG_B5, b5_ckpt),
    ]
    model_probs = []

    for cfg, ckpt in models_cfg:
        model = create_model(cfg["model_name"], cfg["num_classes"])
        model.load_state_dict(torch.load(ckpt))
        model.eval()

        base_ds = TestImageDataset(SAMPLE_SUB, TEST_DIR, get_transform(cfg["img_size"]))
        base_loader = DataLoader(base_ds, batch_size=cfg["batch_size"], shuffle=False)
        probs, ids = infer_probs(model, base_loader)

        tta_sum = np.zeros_like(probs)
        for i in range(cfg["tta_times"]):
            print(f"[{cfg['model_name']}] TTA round {i+1}/{cfg['tta_times']}")
            tta_ds = TestImageDataset(SAMPLE_SUB, TEST_DIR, get_transform(cfg["img_size"], tta=True))
            tta_loader = DataLoader(tta_ds, batch_size=cfg["batch_size"], shuffle=False)
            tta_sum += infer_probs(model, tta_loader)[0]
        final_probs = (probs + tta_sum / cfg["tta_times"]) / 2
        model_probs.append(final_probs)

    # ✅ 3-model Soft voting
    final = sum(model_probs) / len(model_probs)
    preds = final.argmax(1)
    out = pd.DataFrame({"ID": ids, "target": preds})
    path = os.path.join(OUT_DIR, "submission_v13_b4_vit_b5.csv")
    out.to_csv(path, index=False)
    print(f"✅ 최종 결과 저장: {path}")
    return path


# 실행
if __name__ == "__main__":
    resume_main()


🚀 Resume mode: b4 건너뛰고 vit + b5 + ensemble 실행
✅ Oversample 완료: {3: np.int64(12672), 7: np.int64(12928), 14: np.int64(9792)}


/tmp/ipykernel_501159/1902108288.py:6: UserWarning: Argument(s) 'always_apply' are not valid for transform MaxSizeTransform
  A.LongestMaxSize(max_size=img_size, always_apply=True),
/tmp/ipykernel_501159/1902108288.py:7: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=img_size, min_width=img_size,
/tmp/ipykernel_501159/1902108288.py:19: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 40.0), p=0.3),
/tmp/ipykernel_501159/1902108288.py:21: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=70, quality_upper=100, p=0.2),
/tmp/ipykernel_501159/1902108288.py:6: UserWarning: Argument(s) 'always_apply' are not valid for transform MaxSizeTransform
  A.LongestMaxSize(max_size=img_size, always_apply=True),
/tmp/ipykernel_501159/1902108288.py:7: UserWarning: Argument(s) 'value' are not valid for transfo


==== [vit] Epoch 1/40 ====


[Train] loss=1.9235 acc=0.3196 f1=0.3077
[Valid] loss=1.5707 acc=0.4270 f1=0.4035
✅ Best updated: F1=0.40352

==== [vit] Epoch 2/40 ====


[Train] loss=1.3912 acc=0.4950 f1=0.4980
[Valid] loss=1.3509 acc=0.5111 f1=0.5097
✅ Best updated: F1=0.50973

==== [vit] Epoch 3/40 ====


...

[Train] loss=0.0002 acc=1.0000 f1=1.0000
[Valid] loss=0.0001 acc=1.0000 f1=1.0000
⏳ No improve (5/8)

==== [b5] Epoch 37/40 ====


[Train] loss=0.0000 acc=1.0000 f1=1.0000
[Valid] loss=0.0001 acc=1.0000 f1=1.0000
⏳ No improve (6/8)

==== [b5] Epoch 38/40 ====


[Train] loss=0.0001 acc=0.9999 f1=0.9999
[Valid] loss=0.0001 acc=1.0000 f1=1.0000
⏳ No improve (7/8)

==== [b5] Epoch 39/40 ====


[Train] loss=0.0001 acc=1.0000 f1=1.0000
[Valid] loss=0.0001 acc=1.0000 f1=1.0000
⏳ No improve (8/8)
🛑 Early stop

[완료] B4✓ | ViT_F1=0.9892 | B5_F1=1.0000

🔄 3-model Ensemble Inference 시작


NameError: name 'CFG_B5' is not defined

In [18]:
# ============================================================
# 🚀 Inference Only: Use existing B4 / ViT / B5 checkpoints
# ============================================================

import os, torch, numpy as np, pandas as pd
from torch.utils.data import DataLoader

@torch.no_grad()
def ensemble_3model_infer_only():
    print("\n🔄 3-model Ensemble Inference (TTA 포함) 시작")

    # ✅ 이미 학습 완료된 체크포인트 경로 지정
    b4_ckpt = os.path.join(OUT_DIR, "model_b4_best.pt")
    vit_ckpt = os.path.join(OUT_DIR, "model_vit_best.pt")
    b5_ckpt = os.path.join(OUT_DIR, "model_b5_best.pt")

    for p in [b4_ckpt, vit_ckpt, b5_ckpt]:
        if not os.path.exists(p):
            raise FileNotFoundError(f"❌ Checkpoint not found: {p}")

    # ✅ CFG_B5를 전역에 정의 (안돼 있다면)
    CFG_B5 = dict(
        model_name="tf_efficientnet_b5_ns",
        num_classes=17,
        img_size=456,
        batch_size=16,
        lr=3e-4,
        weight_decay=1e-4,
        epochs=40,
        patience=8,
        valid_ratio=0.8,
        num_workers=4,
        tta_times=20,
    )

    # ✅ 세 모델 구성
    models_cfg = [
        (CFG_B4, b4_ckpt),
        (CFG_VIT, vit_ckpt),
        (CFG_B5, b5_ckpt),
    ]
    model_probs = []

    # ✅ 각 모델별 추론 + TTA
    for cfg, ckpt in models_cfg:
        print(f"\n[{cfg['model_name']}] 추론 시작")
        model = create_model(cfg["model_name"], cfg["num_classes"])
        model.load_state_dict(torch.load(ckpt))
        model.eval()

        base_ds = TestImageDataset(SAMPLE_SUB, TEST_DIR, get_transform(cfg["img_size"]))
        base_loader = DataLoader(base_ds, batch_size=cfg["batch_size"], shuffle=False)
        probs, ids = infer_probs(model, base_loader)

        tta_sum = np.zeros_like(probs)
        for i in range(cfg["tta_times"]):
            print(f"[{cfg['model_name']}] TTA round {i+1}/{cfg['tta_times']}")
            tta_ds = TestImageDataset(SAMPLE_SUB, TEST_DIR, get_transform(cfg["img_size"], tta=True))
            tta_loader = DataLoader(tta_ds, batch_size=cfg["batch_size"], shuffle=False)
            tta_sum += infer_probs(model, tta_loader)[0]

        final_probs = (probs + tta_sum / cfg["tta_times"]) / 2
        model_probs.append(final_probs)

    # ✅ Soft voting (3-model 평균)
    print("\n📊 Soft Voting 적용 중...")
    final = sum(model_probs) / len(model_probs)
    preds = final.argmax(1)

    # ✅ 결과 저장
    out = pd.DataFrame({"ID": ids, "target": preds})
    path = os.path.join(OUT_DIR, "submission_v14_b4_vit_b5_infer.csv")
    out.to_csv(path, index=False)
    print(f"✅ 최종 결과 저장: {path}")
    return path


# 실행
if __name__ == "__main__":
    ensemble_3model_infer_only()



🔄 3-model Ensemble Inference (TTA 포함) 시작

[tf_efficientnet_b4_ns] 추론 시작


/data/ephemeral/home/py310/lib/python3.10/site-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(
/tmp/ipykernel_501159/446507045.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don'

[tf_efficientnet_b4_ns] TTA round 1/20


/tmp/ipykernel_501159/1902108288.py:6: UserWarning: Argument(s) 'always_apply' are not valid for transform MaxSizeTransform
  A.LongestMaxSize(max_size=img_size, always_apply=True),
/tmp/ipykernel_501159/1902108288.py:7: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=img_size, min_width=img_size,
/tmp/ipykernel_501159/1902108288.py:29: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 40.0), p=0.3),


[tf_efficientnet_b4_ns] TTA round 2/20


...

/tmp/ipykernel_501159/1902108288.py:6: UserWarning: Argument(s) 'always_apply' are not valid for transform MaxSizeTransform
  A.LongestMaxSize(max_size=img_size, always_apply=True),
/tmp/ipykernel_501159/1902108288.py:7: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=img_size, min_width=img_size,
/tmp/ipykernel_501159/1902108288.py:29: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 40.0), p=0.3),


[tf_efficientnet_b5_ns] TTA round 19/20


/tmp/ipykernel_501159/1902108288.py:6: UserWarning: Argument(s) 'always_apply' are not valid for transform MaxSizeTransform
  A.LongestMaxSize(max_size=img_size, always_apply=True),
/tmp/ipykernel_501159/1902108288.py:7: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=img_size, min_width=img_size,
/tmp/ipykernel_501159/1902108288.py:29: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 40.0), p=0.3),


[tf_efficientnet_b5_ns] TTA round 20/20



📊 Soft Voting 적용 중...
✅ 최종 결과 저장: ./runs_v10_b4_vit_ensemble/submission_v14_b4_vit_b5_infer.csv
